# Big Data Analytics with MovieLens Dataset

In this Jupyter Notebook, we will use the [MovieLens 20M Dataset](https://grouplens.org/datasets/movielens/20m/) on movie ratings to answer several tasks by using `PySpark`. The exercises are structured as a guideline to get familiar with the Pyspark syntax. Have also a look on the [official pySpark documentation](https://spark.apache.org/docs/latest/api/python/pyspark.html). 

**Introduction to Movielens dataset**

The Introduction exercises have the following goals:
- Reading and understanding the schema of our movielens dataset
- Calculating some summary statistics of our dataset
- Learn how to perform joins and aggregations using Spark

This will be also illustrated by guided exercises to get a first understanding of Spark
- Guided Exercise 1: Which movies are the most popular ones?
- Guided Exercise 2: What are the distinct genres in the Movielens Dataset (RDD)?

**Exercises for you:**
- Exercise 1: Which movies have the highest number of ratings?
- Exercise 2: What's the number of movies in each genre?
- Exercise 3: Which movies are a matter of taste?


## Initialize Sparksession

Execute the following cell to initialize a Sparksession:

In [18]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("spark://sparkmaster:7077").appName('movielens').getOrCreate()

## Introduction: Reading the dataset

Our movielens dataset contains 20 million ratings and 465'000 tag applications applied to 27'000 movies by 138'000 users. It also includs tag genome data with 12 million relevance scores across 1100 tags.

The whole dataset contains six CSV files:
- genome-scores.csv
- genome-tags.csv
- links.csv
- movies.csv
- ratings.csv
- tags.csv

In this Introduction exercise, we will have a look on the **`movies`** and **`ratings`** dataframes.

To read a CSV file in our "ml-20m" folder, we access the `DataFrameReader` class through `read` and call the `csv()` method on it. We also specify `option("header", "true")` since the first row of the file contains our column names. 

In [35]:
ratings = spark.read.option("header", "true").csv("ml-20m/ratings.csv")
ratings.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|      2|   3.5|1112486027|
|     1|     29|   3.5|1112484676|
|     1|     32|   3.5|1112484819|
|     1|     47|   3.5|1112484727|
|     1|     50|   3.5|1112484580|
+------+-------+------+----------+
only showing top 5 rows



Each tuple of the `ratings` DataFrame represents one rating (`rating`) for one movie (`movieId`) by one user (`userId`). The ratings ranges from 0.5 stars (worst) up to 5.0 stars (best). 

We can also have look on the Schema of our dataset (column names and types) by using the `printSchema()` method.

In [36]:
ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



Do the same for the `movies.csv` file. What kind of data is available and how does the schema look like?

+-------+----------------------------------+-------------------------------------------+
|movieId|title                             |genres                                     |
+-------+----------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                  |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                    |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)           |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)          |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)|Comedy                                     |
+-------+----------------------------------+-------------------------------------------+
only showing top 5 rows

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



## Guided Exercise 1: Which movies are the most popular ones?

To get the most popular movies, we are looking for the movies with the highest number of ratings. In this task, we assume the number of ratings as a representative for the most popular movies. To do this, we will perform the following *transformations* on the `ratings` DataFrame: 
- group by `movieId`
- count the number of users (`userId`) associated with each movie 
- rename this column to `num_ratings`
- sort by `num_ratings` in descending order 

We do these transformations in `PySpark` and store the DataFrame as `most_popular`. Have also a look on the [official pySpark documentation](https://spark.apache.org/docs/latest/api/python/pyspark.html).

**HINT**:
- Use `agg(count())` to perform an aggregate calculation on grouped data. 
- Don't forget that transformations are [lazy](https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-operations) in spark. We need to call an action (e.g. `show()` for Dataframes, `take()` for RDD's) explicitly to see the results. 

In [38]:
from pyspark.sql.functions import *

most_popular = ratings.groupBy("movieId").agg(count("userId")).withColumnRenamed("count(userId)", "num_ratings").sort(desc("num_ratings"))

most_popular.show(5)

+-------+-----------+
|movieId|num_ratings|
+-------+-----------+
|    296|      67310|
|    356|      66172|
|    318|      63366|
|    593|      63299|
|    480|      59715|
+-------+-----------+
only showing top 5 rows



Unfortunately, the resulting table only contains `movieId` and `num_ratings`. The title of the movie is stored in the `movies` DataFrame. So, we need an inner join of our `most_popular` DataFrame with the `movies` DataFrame on `movieId`.

In [67]:
most_popular_movies = most_popular.join(movies, most_popular.movieId == movies.movieId)
most_popular_movies.select("title", "num_ratings").show(5, truncate = False)

+--------------------------------+-----------+
|title                           |num_ratings|
+--------------------------------+-----------+
|Pulp Fiction (1994)             |67310      |
|Forrest Gump (1994)             |66172      |
|Shawshank Redemption, The (1994)|63366      |
|Silence of the Lambs, The (1991)|63299      |
|Jurassic Park (1993)            |59715      |
+--------------------------------+-----------+
only showing top 5 rows



We now have a list of the most popular (or most rated) movies of our movielens dataset. Have you already watched all of them? :)

## Guided Exercise 2: Understanding RDD (Resilient Distributed Datasets) operations

We will have a look on two core data abstractions of Spark, namely Dataframes and RDD's.
RDDs and DataFrames can be created from external data sources (e.g. HDFS, SQL) or from internal process steps. Dataframes the are easiest abstraction. One can compare Dataframes with a traditional table with columns and rows, which is generally used for handling workflows with structured data. If the data is unstructured (has no schema) and the data needs to be manipulated in non-standard ways, one should use RDD's. Even though our data is structured, we will use some operations on RDD's to understand RDD transformations. 

Have a look on the Pyspark Documentation for RDD operations [PySpark Package](https://spark.apache.org/docs/1.5.1/api/python/pyspark.html)



Execute the following cells and try to understand what map(), flatmap() and take() do

In [40]:
# Load the data into RDD
data = sc.textFile("ml-20m/movies.csv")

# Split the RDD 
moviesRDD = data.map(lambda l: l.split(','))
moviesRDD.take(5)

[['movieId', 'title', 'genres'],
 ['1', 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'],
 ['2', 'Jumanji (1995)', 'Adventure|Children|Fantasy'],
 ['3', 'Grumpier Old Men (1995)', 'Comedy|Romance'],
 ['4', 'Waiting to Exhale (1995)', 'Comedy|Drama|Romance']]

In [41]:
# map
Reversed = moviesRDD.map(lambda m: m[::-1])
Reversed.take(5)

[['genres', 'title', 'movieId'],
 ['Adventure|Animation|Children|Comedy|Fantasy', 'Toy Story (1995)', '1'],
 ['Adventure|Children|Fantasy', 'Jumanji (1995)', '2'],
 ['Comedy|Romance', 'Grumpier Old Men (1995)', '3'],
 ['Comedy|Drama|Romance', 'Waiting to Exhale (1995)', '4']]

In [42]:
# flatMap
words = moviesRDD.flatMap(lambda m: list(m))
words.take(10)

['movieId',
 'title',
 'genres',
 '1',
 'Toy Story (1995)',
 'Adventure|Animation|Children|Comedy|Fantasy',
 '2',
 'Jumanji (1995)',
 'Adventure|Children|Fantasy',
 '3']

## Extracting Distinct Movie Genres

Now we will try to combine these operations above by trying to extract all distinct movie genres in our Movielens data. You need to:
- Read the csv file located in ("ml-20m/movies.csv")
- Split the data and select the corresponding genre column via `map()`
- `flatmap()` the data --> **Remark**: Be aware that a movie can contain several genres delimited by `('|')`
- Print the results by`take()` the `distinct()` genres


In [43]:
movies_rdd=sc.textFile("ml-20m/movies.csv")

movies_split= movies_rdd.map(lambda lines:lines.split(','))
genres=movies_split.map(lambda line:(line[2]))
genre=genres.flatMap(lambda l:l.split('|'))
genres_distinct=genre.distinct()

print(genres_distinct.take(10))

['Children', 'Fantasy', 'Romance', 'Drama', 'Action', 'Thriller', ' The (1995)"', 'Horror', 'Sci-Fi', 'IMAX']


## Exercise 1: Which movies have the highest ratings (in average)?

Now it's your turn. We want to see which movies are rated to be the best. You need to use the `ratings` DataFrame and: 

- Group by `movieId` 
- Calculate the average rating for each movie and rename this column to `avg_rating`
- Sort by `avg_rating` in descending order 
- Join the resulting DataFrame with the `movies` DataFrame to get the movienames.


**NOTE** Be sure that you read the movies file

In [69]:
top_rated = 

+-------+-----------+----------+-------+---------------------------------------------------------+-----------+
|movieId|num_ratings|avg_rating|movieId|title                                                    |genres     |
+-------+-----------+----------+-------+---------------------------------------------------------+-----------+
|108527 |2          |5.0       |108527 |Catastroika (2012)                                       |Documentary|
|103871 |2          |5.0       |103871 |Consuming Kids: The Commercialization of Childhood (2008)|Documentary|
|111548 |1          |5.0       |111548 |Welcome to Australia (1999)                              |Documentary|
|95517  |1          |5.0       |95517  |Barchester Chronicles, The (1982)                        |Drama      |
|109715 |1          |5.0       |109715 |Inquire Within (2012)                                    |Comedy     |
+-------+-----------+----------+-------+---------------------------------------------------------+-----------+
o

You will realize that all of the movies with `avg_rating` of exactly 5.0 have 2 or less `num_ratings` . We must investigate the distribution of `num_ratings` to only consider movies that have a minimum number of ratings. Calculate some summary statistics within Spark (consider `mean()`, `min()` and `max()`) and take a decent value to filter your results:

+-------+-----------+-----------------+-------+--------------------+--------------------+
|movieId|num_ratings|       avg_rating|movieId|               title|              genres|
+-------+-----------+-----------------+-------+--------------------+--------------------+
|    318|      63366|4.446990499637029|    318|Shawshank Redempt...|         Crime|Drama|
|    858|      41355|4.364732196832306|    858|Godfather, The (1...|         Crime|Drama|
|     50|      47006|4.334372207803259|     50|Usual Suspects, T...|Crime|Mystery|Thr...|
|    527|      50054|4.310175010988133|    527|Schindler's List ...|           Drama|War|
|   1221|      27398|4.275640557704942|   1221|Godfather: Part I...|         Crime|Drama|
+-------+-----------+-----------------+-------+--------------------+--------------------+
only showing top 5 rows



## Exercise 2: What's the number of movies in each genre?

In this exercise, we want to calculate the number of movies in each genre. This exercise is similar to the guided introduction exercise 2:

- Read the csv file located in ("ml-20m/movies.csv")
- Split the data and select the corresponding genre column via `map()`
- `flatmap()` the data --> **Remark**: Be aware that a movie can contain several genres delimited by `('|')`
- Have a look on the [official pySpark documentation](https://spark.apache.org/docs/latest/api/python/pyspark.html) and check what the `reduceByKey()` function do. This function is needed to sum up the number of movies in each genre. 
- Sort the results using the `sortBy()` function
- Print the results by`take()`

In [52]:
movies_rdd =

[('Drama', 9952), ('Comedy', 6545), ('Romance', 3186), ('Thriller', 3131), ('Action', 2851), ('Crime', 2251), ('Documentary', 1979), ('Horror', 1949), ('Adventure', 1750), ('Sci-Fi', 1394)]


## Exercise 3: Which movies are a matter of taste?

As you know, movies are a matter of taste. There are for sure some movies, which you would rate with a 5 whereas your friend rates the same movie with a 2. These are the movies that divide your opinon. Try to find out, which movies belong to this category.

**HINT**

- We need to consider the standard deviation of the movie ratings
- Also, try to consider only movies that have some minimum number of ratings (e.g. 700) 
- Join with the movies table to get the movie names

In [64]:
matterofTaste_movies = 

+-----------------------------------+-----------+------------------+
|title                              |num_ratings|std_rating        |
+-----------------------------------+-----------+------------------+
|Plan 9 from Outer Space (1959)     |2304       |1.420171182322382 |
|Freddy Got Fingered (2001)         |1550       |1.3504497277537106|
|Twilight Saga: Eclipse, The (2010) |884        |1.3485057430514158|
|Twilight Saga: New Moon, The (2009)|1135       |1.3367548401080391|
|Passion of the Christ, The (2004)  |3130       |1.335427370705759 |
+-----------------------------------+-----------+------------------+
only showing top 5 rows

